<a href="https://colab.research.google.com/github/fambargh/SAMPLE/blob/master/fine_tune2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
cd /content

/content


In [0]:
! unzip fine-tuning-keras.zip
#!cd fine-tuning-keras

In [0]:
! mkdir /content/fine-tuning-keras/Food-11


In [0]:
%cd /content/fine-tuning-keras/Food-11
!unzip classification1.zip

In [3]:
%cd /content/fine-tuning-keras/dataset

/content/fine-tuning-keras/dataset


In [0]:
rm -rf train

In [4]:
cd /content/fine-tuning-keras

/content/fine-tuning-keras


In [6]:
! python build_dataset.py 

[INFO] processing 'train split'...
[INFO] processing 'test split'...
[INFO] processing 'valid split'...


In [7]:
ls /content/fine-tuning-keras/dataset/

test/  train/  valid/


In [8]:
ls  /content/fine-tuning-keras/dataset/train/

0/  1/


In [9]:
ls -l /content/fine-tuning-keras/dataset/train/0/*.png | head -n 5

-rw-r--r-- 1 root root 23482 Dec 20 23:46 /content/fine-tuning-keras/dataset/train/0/0.drishtiGS_041.png
-rw-r--r-- 1 root root 24054 Dec 20 23:46 /content/fine-tuning-keras/dataset/train/0/0.drishtiGS_042.png
-rw-r--r-- 1 root root 22324 Dec 20 23:46 /content/fine-tuning-keras/dataset/train/0/0.drishtiGS_046.png
-rw-r--r-- 1 root root 23259 Dec 20 23:46 /content/fine-tuning-keras/dataset/train/0/0.drishtiGS_047.png
-rw-r--r-- 1 root root 20738 Dec 20 23:46 /content/fine-tuning-keras/dataset/train/0/0.drishtiGS_057.png


In [14]:
! python train.py

Using TensorFlow backend.
Found 182 images belonging to 2 classes.
Found 39 images belonging to 2 classes.
Found 39 images belonging to 2 classes.







2019-12-24 19:22:41.603259: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX512F
2019-12-24 19:22:41.608022: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2000179999 Hz
2019-12-24 19:22:41.608206: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x5f7db80 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2019-12-24 19:22:41.608256: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2019-12-24 19:22:41.610039: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2019-12-24 19:22:41.706674: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful N

In [19]:
!python predict.py --image '/content/fine-tuning-keras/Food-11/test/0/0.drishtiGS_007.png'

Using TensorFlow backend.
[INFO] loading model...





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.



2019-12-24 19:42:14.027557: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX512F
2019-12-24 19:42:14.032721: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2000179999 Hz
2019-12-24 19:42:14.032936: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7a66840 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2019-12-24 19:42:14.032987: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2019-12-24 19:42:14.035427: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2019-12-24 19:42:14.139660: I tensorflow/stream_executor/cuda/cuda_gpu_executor.c

In [0]:
!zip -r /content/fine_tune_me.zip  /content/fine-tuning-keras

In [22]:
"""
Author: Abner Ayala-Acevedo
This script based on examples provided in the keras documentation and a blog.
"Building powerful image classification models using very little data"
from blog.keras.io.
Dataset: Subset of Kaggle Dataset
https://www.kaggle.com/c/dogs-vs-cats/data
- cat pictures index 0-999 in data/train/cats
- cat pictures index 1000-1400 in data/validation/cats
- dogs pictures index 0-999 in data/train/dogs
- dog pictures index 1000-1400 in data/validation/dogs
Example: Dogs vs Cats (Directory Structure)
data/
    train/
        dogs/
            dog001.jpg
            dog002.jpg
            ...
        cats/
            cat001.jpg
            cat002.jpg
            ...
    validation/
        dogs/
            dog001.jpg
            dog002.jpg
            ...
        cats/
            cat001.jpg
            cat002.jpg
            ...
Example has 1000 training examples for each class, and 400 validation examples for each class.
The data folder already contains the dogs vs cat data you simply need to run script. For the dogs_cats classification
you can find a model already trained in the model folder. Feel free to create your own data.
"""
import numpy as np
import sys
import os
from keras.layers import *
from keras.optimizers import *
from keras.applications import *
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras import backend as k
import keras

# fix seed for reproducible results (only works on CPU, not GPU)
seed = 9
np.random.seed(seed=seed)
tf.set_random_seed(seed=seed)

# hyper parameters for model
nb_classes = 2  # number of classes
based_model_last_block_layer_number = 126  # value is based on based model selected.
img_width, img_height = 299, 299  # change based on the shape/structure of your images
batch_size = 32  # try 4, 8, 16, 32, 64, 128, 256 dependent on CPU/GPU memory capacity (powers of 2 values).
nb_epoch = 50  # number of iteration the algorithm gets trained.
learn_rate = 1e-4  # sgd learning rate
momentum = .9  # sgd momentum to avoid local minimum
transformation_ratio = .05  # how aggressive will be the data augmentation/transformation
nb_train_samples = 2000  # Total number of train samples. NOT including augmented images
nb_validation_samples = 800  # Total number of train samples. NOT including augmented images.


def train(train_data_dir, validation_data_dir, model_path):
    # Pre-Trained CNN Model using imagenet dataset for pre-trained weights
    # base_model = Xception(input_shape=(img_width, img_height, 3), weights='imagenet', include_top=False)
    
    if K.image_data_format() == 'channels_first':
        input_shape = (3, img_width, img_height)
    else:
        input_shape = (img_width, img_height, 3)
    
    base_model = InceptionV3(input_shape=input_shape, weights='imagenet', include_top=False)

    # # Top Model Block
    # x = base_model.output
    # x = GlobalAveragePooling2D()(x)
    # predictions = Dense(nb_classes, activation='softmax')(x)
    
    # add a global spatial average pooling layer
    # x = base_model.output
    # x = GlobalAveragePooling2D()(x)
    # # let's add a fully-connected layer
    # x = Dense(1024, activation='relu', name='fc1')(x)
    
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(256, activation='relu', name='fc1')(x)
    x = Dropout(0.5)(x)

    predictions = Dense(nb_classes, activation='softmax', name='predictions')(x)

    # add your top layer block to your base model
    model = Model(base_model.input, predictions)
    print(model.summary())

    # # let's visualize layer names and layer indices to see how many layers/blocks to re-train
    # # uncomment when choosing based_model_last_block_layer
    # for i, layer in enumerate(model.layers):
    #     print(i, layer.name)

    # first: train only the top layers (which were randomly initialized)
    # i.e. freeze all layers of the based model that is already pre-trained.
    for layer in base_model.layers:
        layer.trainable = False

    # Read Data and Augment it: Make sure to select augmentations that are appropriate to your images.
    # To save augmentations un-comment save lines and add to your flow parameters.
    train_datagen = ImageDataGenerator(rescale=1. / 255,
                                       rotation_range=transformation_ratio,
                                       shear_range=transformation_ratio,
                                       zoom_range=transformation_ratio,
                                       cval=transformation_ratio,
                                       horizontal_flip=True,
                                       vertical_flip=True)

    validation_datagen = ImageDataGenerator(rescale=1. / 255)

    # os.makedirs(os.path.join(os.path.abspath(train_data_dir), 'preview'), exist_ok=True)
    train_generator = train_datagen.flow_from_directory(train_data_dir,
                                                        target_size=[img_width, img_height],
                                                        batch_size=batch_size,
                                                        class_mode='categorical')
    # save_to_dir=os.path.join(os.path.abspath(train_data_dir), '../preview')
    # save_prefix='aug',
    # save_format='jpeg')
    # use the above 3 commented lines if you want to save and look at how the data augmentations look like

    validation_generator = validation_datagen.flow_from_directory(validation_data_dir,
                                                                  target_size=[img_width, img_height],
                                                                  batch_size=batch_size,
                                                                  class_mode='categorical')

    model.compile(optimizer='nadam',
                  loss='categorical_crossentropy',  # categorical_crossentropy if multi-class classifier
                  metrics=['accuracy'])

    # save weights of best training epoch: monitor either val_loss or val_acc

    top_weights_path = os.path.join(os.path.abspath(model_path), 'top_model_weights.h5')
    callbacks_list = [
        ModelCheckpoint(top_weights_path, monitor='val_acc', verbose=1, save_best_only=True),
        EarlyStopping(monitor='val_acc', patience=5, verbose=0),
        keras.callbacks.TensorBoard(log_dir='tensorboard/inception-v3-train-top-layer', histogram_freq=0, write_graph=False, write_images=False)
    ]

    # Train Simple CNN
    model.fit_generator(train_generator,
                        steps_per_epoch=nb_train_samples // batch_size,
                        epochs=nb_epoch / 5,
                        validation_data=validation_generator,
                        validation_steps=nb_validation_samples // batch_size,
                        callbacks=callbacks_list)

    # verbose
    print("\nStarting to Fine Tune Model\n")

    # add the best weights from the train top model
    # at this point we have the pre-train weights of the base model and the trained weight of the new/added top model
    # we re-load model weights to ensure the best epoch is selected and not the last one.
    model.load_weights(top_weights_path)

    # based_model_last_block_layer_number points to the layer in your model you want to train.
    # For example if you want to train the last block of a 19 layer VGG16 model this should be 15
    # If you want to train the last Two blocks of an Inception model it should be 172
    # layers before this number will used the pre-trained weights, layers above and including this number
    # will be re-trained based on the new data.
    for layer in model.layers[:based_model_last_block_layer_number]:
        layer.trainable = False
    for layer in model.layers[based_model_last_block_layer_number:]:
        layer.trainable = True

    # compile the model with a SGD/momentum optimizer
    # and a very slow learning rate.
    model.compile(optimizer='nadam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
                  
    
    # save weights of best training epoch: monitor either val_loss or val_acc
    final_weights_path = os.path.join(os.path.abspath(model_path), 'model_weights.h5')
    callbacks_list = [
        ModelCheckpoint(final_weights_path, monitor='val_acc', verbose=1, save_best_only=True),
        EarlyStopping(monitor='val_loss', patience=5, verbose=0),
        keras.callbacks.TensorBoard(log_dir='tensorboard/inception-v3-fine-tune', histogram_freq=0, write_graph=False, write_images=False)
    ]
    



    # fine-tune the model
    model.fit_generator(train_generator,
                        steps_per_epoch=nb_train_samples // batch_size,
                        epochs=nb_epoch,
                        validation_data=validation_generator,
                        validation_steps=nb_validation_samples // batch_size,
                        callbacks=callbacks_list)

    # save model
    model_json = model.to_json()
    with open(os.path.join(os.path.abspath(model_path), 'model.json'), 'w') as json_file:
        json_file.write(model_json)


train('/content/fine-tuning-keras/Food-11/train', '/content/fine-tuning-keras/Food-11/valid', '.')

# if __name__ == '__main__':
#     if not len(sys.argv) == 3:
#         print('Arguments must match:\npython code/fine_tune.py <data_dir/> <model_dir/>')
#         print('Example: python code/fine_tune.py data/dogs_cats/ model/dog_cats/')
#         sys.exit(2)
#     else:
#         data_dir = os.path.abspath(sys.argv[1])
#         train_dir = os.path.join(os.path.abspath(data_dir), 'train')  # Inside, each class should have it's own folder
#         validation_dir = os.path.join(os.path.abspath(data_dir), 'validation')  # each class should have it's own folder
#         model_dir = os.path.abspath(sys.argv[2])

#         os.makedirs(os.path.join(os.path.abspath(data_dir), 'preview'), exist_ok=True)
#         os.makedirs(model_dir, exist_ok=True)

#     train(train_dir, validation_dir, model_dir)  # train model

#     # release memory
#     k.clear_session()














87916544/87910968 [==============================] - 4s 0us/step
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 149, 149, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 149, 149, 32) 96          conv2d_1[0][0]                   
_________________________________________________________________________________________________

In [31]:
# USAGE
# python predict.py --image dataset/evaluation/Egg/3_137.jpg 

# import the necessary packages
from keras.models import load_model
#from pyimagesearch import config
import numpy as np
import argparse
import imutils
import cv2

# construct the argument parser and parse the arguments
#ap = argparse.ArgumentParser()
#ap.add_argument("-i", "--image", type=str, required=True,
	#help="path to our input image")
#args = vars(ap.parse_args())

# load the input image and then clone it so we can draw on it later
image = cv2.imread("/content/fine-tuning-keras/Food-11/test/1/1.G-11-R.png")
output = image.copy()
output = imutils.resize(output, width=400)

# our model was trained on RGB ordered images but OpenCV represents
# images in BGR order, so swap the channels, and then resize to
# 224x224 (the input dimensions for VGG16)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
image = cv2.resize(image, (299, 299))

# convert the image to a floating point data type and perform mean
# subtraction
image = image.astype("float32")
mean = np.array([123.68, 116.779, 103.939][::1], dtype="float32")
image -= mean

# load the trained model from disk
print("[INFO] loading model...")
model = load_model('/content/fine-tuning-keras/top_model_weights.h5')

# pass the image through the network to obtain our predictions
preds = model.predict(np.expand_dims(image, axis=0))[0]
i = np.argmax(preds)
#label = class_mode[i]

# draw the prediction on the output image
text = " {:.2f}%".format(preds[i] * 100)
cv2.putText(output, text, (3, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5,
	(0, 255, 0), 2)

# show the output image
cv2.imwrite('/content/output.png',output)
#cv2.imshow("Output", output)
#cv2.waitKey(0)

[INFO] loading model...


True